# Problem Setup

As input, we are given a graph $G = (V_G, E_G)$, where each vertex is a geographic position $s_i \in S^2$, and each edge $(i, j)$ has an associated (Olivier-Ricci) curvature $\kappa_{i, j} \in (-2, 1)$ and an associated latency $t_{i, j} \in \mathbb{R}_{\ge 0}$.

Intuitively, we want to return a surface in $\mathbb{R}^3$ that is the graph of a function $f : S^2 \to \mathbb{R}_{> 0}$ whose geodesics $g_{i, j}$ between $s_i$ and $s_j$ (and their missing $\rho$-coordinates) have length $\phi_{i, j}$ that is in a linear relationship with the latency.

The strategy to realize this intuition is to create a mesh $M = (V_M, E_M)$ supported on a subset of $S^2$ that contains our input positions. Let $P$ be the support. Then for each $s_i \in P$, we want to assign a $\rho_i \in \mathbb{R}_{> 0}$, which in turn gives a point $v_i = (s_i, \rho_i) \in V$. This setup is made explicit in `mesh/sphere.ipynb`.

In [1]:
from pathlib import Path
import sys
path = Path().resolve()
for crumb in __name__.split('.'):
    path = path / crumb
sys.path.insert(0, str((path / '..' / '..').resolve()))

In [2]:
import itertools

import numpy as np
from scipy import linalg

from linear_geodesic_optimization.utils import notebook_import
from linear_geodesic_optimization.mesh.mesh import Animation3D
from linear_geodesic_optimization.mesh.sphere import Mesh as SphereMesh
from linear_geodesic_optimization.optimization import laplacian, geodesic, linear_regression, smooth

# Objective/Loss Functions

To enforce that the mesh approximates our desired surface, we define the objective functions $$\begin{aligned}
    \mathcal{L}_{\mathrm{geodesic}}(M) &\triangleq \sum_{e \in E_G} (\text{least squares residual of edge \(e\)})^2, \\
    \mathcal{L}_{\mathrm{smooth}}(M) &\triangleq -\rho^\intercal L_C\rho, \\
    \mathcal{L}(M) &\triangleq \mathcal{L}_{\mathrm{geodesic}}(M) + \lambda\mathcal{L}_{\mathrm{smooth}}(M),
\end{aligned}$$ where $\lambda$ is a tunable hyperparameter. Our goal is then to minimize $\mathcal{L}(M)$.

Note that the loss functions (particularly the geodesic and total ones) also have a dependence on the measured latencies. We omit that as a written parameter because they are treated as fixed (we are really optimizing over the manifold, not over the "book" measurements).

# Putting It All Together with Minibatch Gradient Descent

We can rewrite $$\mathcal{L}_{\text{geodesic}}(M) = \sum_{v \in V_G} \sum_{\substack{v' \in V_G \\ (v, v') \in E_G}} \frac{1}{2}(\text{least squares residual of edge \((v, v')\)})^2.$$ From here, we can take the standard approach of batching the gradient computation by vertices. This idea fits well with the heat method, since the heat method necessarily computes all geodesic distances (and their gradients) from a single source (represented by $v$ in the above decomposition).

Motivated by this, define $$\mathcal{L}_{\text{geodesic}, v}(M) = \sum_{\substack{v' \in V_G \\ (v, v') \in E_G}} \frac{1}{2}(\text{least squares residual of edge \((v, v')\)})^2.$$

## Partial Derivative Selection

Computing the gradient of $\mathcal{L}_{\text{geodesic}, v}(M)$ is quite expensive. We can make the computation more efficient by approximately reconstructing the geodesics from $v$. Importantly, for vertices not on the faces through which the geodesics pass, the partial derivatives will be $0$.

### Fixed Point Iteration Strategy

Consider the set of geodesics between $v_a$ and $v_b$ for $b \in B$, where the geodesic distance is computed with $\gamma = \{v_a\}$. A conservative estimate of these faces incident to these geodesics can be constructed iteratively by finding a minimal fixed point $S_{a, B}$ of $$S \mapsto \left\{(v_i \to v_j \to v_{c(i, j)}) : \begin{array}{c}\text{$(v_i \to v_j \to v_{c(i, j)})$ is adjacent to a face $(v_j \to v_i \to v_{c(j, i)}) \in S$} \\ \text{and $\phi^\gamma_{c(i, j)} < \max(\phi^\gamma_i, \phi^\gamma_j)$}\end{array}\right\}$$ such that $$\{(v_b \to v_i \to v_{c(b, i)}) : \phi^\gamma_b > \min(\phi^\gamma_i, \phi^\gamma_{c(b, i)}), b \in B\} \subseteq S_{a, B}.$$ In other words, $S_{a, B}$ is (at least) the set of faces through which paths that always move towards $v_a$ can pass through.

With this computed, the vertices we care about are those incident to the faces in $S_{a, B}$.

In [3]:
def approximate_geodesics_fpi(mesh, phi, initial_vertices):
    e = mesh.get_edges()
    c = mesh.get_c()
    vertices = set()
    to_process = []
    processed = set()
    for b in initial_vertices:
        for i in e[b]:
            cbi = c[b,i]
            if phi[b] > phi[i] or phi[b] > phi[cbi]:
                to_process.append((b, i, cbi))

    while to_process:
        (i, j, k) = to_process[-1]
        del to_process[-1]
        if j < i and j < k:
            j, k, i = i, j, k
        elif k < i and k < j:
            k, i, j = i, j, k
        if (i, j, k) in processed:
            continue

        vertices.add(i)
        vertices.add(j)
        vertices.add(k)

        cji = c[j,i]
        if phi[cji] < phi[i] or phi[cji] < phi[j]:
            to_process.append((j, i, cji))

        ckj = c[k,j]
        if phi[ckj] < phi[j] or phi[ckj] < phi[k]:
            to_process.append((k, j, ckj))

        cik = c[i,k]
        if phi[cik] < phi[k] or phi[cik] < phi[i]:
            to_process.append((i, k, cik))

        processed.add((i, j, k))

    return vertices


### Triangle Inequality Strategy

Suppose we have the same setup as before, where we want to find the faces incident to the geodesic between $v_a$ and $v_b$. Let's focus in on a face $f = (v_i \to v_j \to v_{c(i, j)})$. Note that geodesic distance satisfies the triangle inequality. Define $$\begin{aligned}
    \widetilde{a} \in \argmin_{\ell \in \{i, j, c(i, j)\}}\left(\phi^{\{v_a\}}_\ell\right), \\
    \widetilde{b} \in \argmin_{\ell \in \{i, j, c(i, j)\}}\left(\phi^{\{v_b\}}_\ell\right).
\end{aligned}$$ From the triangle inequality, we have $$\phi^{\{v_a\}}_b \le \phi^{\{v_a\}}_{\widetilde{a}} + \|v_{\widetilde{a}} - v_{\widetilde{b}}\|_2 + \phi^{\{v_b\}}_{\widetilde{b}}.$$ Intuitively, this inequality is saying that $f$ cannot be too far away from $v_a$ and $v_b$.

Note that any $f$ that satisfies the above also automatically satisfies the conditions of the fixed point strategy (the "monotonic" path is the concatenation of the geodesic from $v_b$ to $v_{\widetilde{b}}$, the edge from $v_{\widetilde{b}}$ to $v_{\widetilde{a}}$ (or no edge if $\widetilde{b} = \widetilde{a}$), and the geodesic from $v_{\widetilde{a}}$ to $v_a$). Therefore, this strategy is a refinement of the previous one. Unfortunately, the previous strategy required thinking only of $\phi^{\{v_a\}}$, whereas this strategy requires considering additionally $\phi^{\{v_b\}}$ for each $b$ such that $(a, b) \in E_G$.

In [4]:
# TODO: Implement the triangle inequality strategy


In [5]:
# Construct the mesh
frequency = 2
M = SphereMesh(frequency)
rng = np.random.default_rng()
directions = M.get_directions()
V = directions.shape[0]
dif_v = {l: directions[l] for l in range(V)}
rho = rng.random(V) + 0.5
rho /= sum(linalg.norm(rho[l]) for l in range(V)) / V
M.set_rho(rho)


In [6]:
# Get some (phony) latency measurements
lat_long_pairs = [
    (0, 0),
    (0, 90),
    (90, 0),
    (0, 180),
    (0, -90),
    (-90, 0),
]
directions = [SphereMesh.latitude_longitude_to_direction(lat, long) for (lat, long) in lat_long_pairs]
s_indices = [M.nearest_direction_index(direction) for direction in directions]
ts = {si: [(sj, np.arccos(dsi @ dsj))
           for j, (sj, dsj) in enumerate(zip(s_indices, directions)) if (i - j) % 6 != 0]
      for i, (si, dsi) in enumerate(zip(s_indices, directions))}


In [7]:
# Construct the differentiation heirarchy
laplacian_forward = laplacian.Forward(M)
geodesic_forward = geodesic.Forward(M, laplacian_forward)
linear_regression_forward = linear_regression.Forward()
smooth_forward = smooth.Forward(M)
laplacian_reverse = laplacian.Reverse(M, laplacian_forward)
geodesic_reverse = geodesic.Reverse(M, geodesic_forward, laplacian_reverse)
linear_regression_reverse = linear_regression.Reverse(linear_regression_forward)
smooth_reverse = smooth.Reverse(M)


In [8]:
# Run gradient descent

lam = 0.1

def get_losses(s_indices, ts):
    lse = 0
    for s_index in np.random.permutation(s_indices):
        gamma = [s_index]
        s_connected, t = zip(*ts[s_index])
        s_connected = list(s_connected)
        t = np.array(t)
        phi = geodesic_forward.calc_phi(gamma)
        lse += linear_regression_forward.calc_lse(phi[s_connected], t)
    L_smooth = smooth_forward.calc_L_smooth()
    return lse, L_smooth

rng = np.random.default_rng()

animation_3D = Animation3D()

max_iterations = 2
for i in itertools.count(1):
    if i > max_iterations:
        break

    eta = 1 / i

    lse, L_smooth = get_losses(s_indices, ts)
    print(f'iteration {i}: \n\tlse: {lse:.6f}\n\tL_smooth: {L_smooth:.6f}\n\tLoss: {(lse + lam * L_smooth):.6f}')

    for s_index in np.random.permutation(s_indices):
        animation_3D.add_frame(M)
        gamma = [s_index]
        s_connected, t = zip(*ts[s_index])
        s_connected = list(s_connected)
        t = np.array(t)

        dif_L = np.zeros(V)
        dif_L_smooth = smooth_reverse.calc_dif_L_smooth(dif_v)
        for l in range(V):
            dif_L[l] +=  lam * dif_L_smooth[l]

        phi = geodesic_forward.calc_phi(gamma)
        ls = approximate_geodesics_fpi(M, phi, s_connected)
        dif_phi = geodesic_reverse.calc_dif_phi(gamma, dif_v, ls)
        dif_phi = {l: dif[s_connected] for l, dif in dif_phi.items()}
        dif_lse = linear_regression_reverse.calc_dif_lse(phi[s_connected], t, dif_phi, ls)

        for l in dif_lse:
            dif_L[l] += dif_lse[l]

        rho = np.maximum(rho - eta * dif_L, 0.01)
        rho /= sum(linalg.norm(rho[l]) for l in range(V)) / V
        M.set_rho(rho)

lse, L_smooth = get_losses(s_indices, ts)
print(f'\nFinal lse: {lse:.6f}\nFinal L_smooth: {L_smooth:.6f}\nFinal Loss: {(lse + lam * L_smooth):.6f}')
animation_3D.add_frame(M)


iteration 1: 
	lse: 1.635265
	L_smooth: 9.453518
	Loss: 2.580616
iteration 2: 
	lse: 0.024401
	L_smooth: 0.050427
	Loss: 0.029444

Final lse: 0.007927
Final L_smooth: 0.011996
Final Loss: 0.009126


In [9]:
animation_3D.get_fig(duration=50).show()

# The following values should hopefully both be near 1.0
print(np.min(rho), np.max(rho))


0.9566756632243263 1.0340793787583795


TODO:
* ~~Descend from a noisy sphere (or other simple surfaces) to a "clean" one~~
* ~~Add visualization of GD~~
* Try nonuniform meshes (e.g., more detail in America, less in the oceans)
* ~~Try running with $\lambda = 0$ (and see what happens with different initializations)~~
* Is it possible to first optimize over geodesics, and then optimize over smoothness? (My gut instinct is no, but maybe it could be possible under certain circumstances)

Next steps:
* Other smoothness functions
* What's a good value for $\eta$? Just looking for something that makes the output "pretty good"
* Same with $\lambda$
* Later step is to plot the graph representing the network on the sphere